In [7]:
import pandas as pd
import numpy as np
import math
from datetime import timedelta
import statsmodels.api as sm
from datetime import datetime

In [8]:
# Reading the synthetic rooftop PV dataset and site details file associated with it
# (Important Note: that the results produced using the dummy dataset are likely to be different with the ones
# shown in the research article)

input_df = pd.read_csv("Input_data/synthetic_rooftop_data.gz", compression='gzip')
input_df.index = pd.to_datetime(input_df["t_stamp_utc"])
input_df = input_df.drop(columns=["t_stamp_utc"])
site_details_df = pd.read_csv("Input_data/site_details.csv", index_col=0)
all_sites = site_details_df.index

In [9]:
# Reads the predictability values of the actual rooftop PV dataset based on different potential metrics

# Setting the time series resampling intervals and the WPE/PE hyperparameter:
resampling_interval_list = ['5min', '10min', '15min', '20min', '25min', '30min']
dimension_list = [3, 4, 5, 6]

entropy_df = pd.DataFrame()
entropy_df.index = all_sites

for resampling_interval in resampling_interval_list:
    for dimension in dimension_list:
        entropy_df['PE' + str(dimension) + '_' + resampling_interval] = ''
        for site_id in all_sites:
            PE_vals = pd.read_pickle('Processed_data/PEvalues_' + resampling_interval +
                                     'minresamp_' + str(dimension) + 'dim.pkl')
            entropy_df.loc[site_id, 'PE' + str(dimension) + '_' + resampling_interval] \
                = PE_vals.loc[PE_vals['ID'] == site_id]['PEVal'].values

for resampling_interval in resampling_interval_list:
    for dimension in dimension_list:
        entropy_df['WPE' + str(dimension) + '_' + resampling_interval] = ''
        for site_id in all_sites:
            WPE_vals = pd.read_pickle('Processed_data/WPEvalues_' + resampling_interval +
                                      'minresamp_' + str(dimension) + 'dim.pkl')
            entropy_df.loc[site_id, 'WPE' + str(dimension) + '_' + resampling_interval] \
                = WPE_vals.loc[WPE_vals['ID'] == site_id]['WPEVal'].values

for resampling_interval in resampling_interval_list:
    entropy_df['SpE' + '_' + resampling_interval] = ''
    for site_id in all_sites:
        SpE_vals = pd.read_pickle('Processed_data/SpEvalues_' + resampling_interval + '.pkl')
        entropy_df.loc[site_id, 'SpE' + '_' + resampling_interval] \
            = SpE_vals.loc[SpE_vals['ID'] == site_id]['SpEVal'].values

In [11]:
# Calculates the median/mean of prediction errors of PV generation data for minutes ahead prediction

# Setting the forecast horizons that are of importance in our application:
# (shortest horizon is assumed to be the data temporal resolution)
forecast_horizon_list = ['5min', '10min', '15min', '20min']

train_hours = 3

datapoint_in_eachhour = int(timedelta(hours=1) /
                            timedelta(minutes=datetime.strptime(forecast_horizon_list[0], '%Mmin').minute))

test_data_length = int(timedelta(minutes=datetime.strptime(forecast_horizon_list[-1], '%Mmin').minute) /
                       timedelta(minutes=datetime.strptime(forecast_horizon_list[0], '%Mmin').minute))

train_data_length = train_hours * datapoint_in_eachhour

number_of_predictions = 24 * datapoint_in_eachhour * 365 - train_data_length - test_data_length

forecast_error_df = pd.DataFrame()
forecast_error_df.index = all_sites

for forecast_horizon in forecast_horizon_list:
    print(forecast_horizon)
    ARIMA_MAE_df = pd.read_csv('Processed_data/ARIMA_' + str(train_hours) + 'hr_' + forecast_horizon + '_MAE.xz',
                                compression='xz', index_col=0)
    naive_MAE_df = pd.read_csv('Processed_data/naive_' + str(train_hours) + 'hr_' + forecast_horizon + '_MAE.xz',
                                compression='xz', index_col=0)
    ARIMA_MSE_df  = pd.read_csv('Processed_data/ARIMA_' + str(train_hours) + 'hr_' + forecast_horizon + '_MSE.xz',
                                compression='xz', index_col=0)
    naive_MSE_df = pd.read_csv('Processed_data/naive_' + str(train_hours) + 'hr_' + forecast_horizon + '_MSE.xz',
                                compression='xz', index_col=0)
    for site_id in input_df.columns:
        forecast_error_df.loc[site_id, 'NMAE_ARIMA_' + forecast_horizon] = \
            ARIMA_MAE_df[site_id].sum()/(number_of_predictions*np.percentile(input_df[site_id], 99))
        forecast_error_df.loc[site_id, 'NMAE_naive_' + forecast_horizon] = \
            naive_MAE_df[site_id].sum()/(number_of_predictions*np.percentile(input_df[site_id], 99))
        forecast_error_df.loc[site_id, 'NRMSE_ARIMA_' + forecast_horizon] = \
            math.sqrt(ARIMA_MSE_df[site_id].sum()/number_of_predictions)/np.percentile(input_df[site_id], 99)
        forecast_error_df.loc[site_id, 'NRMSE_naive_' + forecast_horizon] = \
            math.sqrt(naive_MSE_df[site_id].sum()/number_of_predictions)/np.percentile(input_df[site_id], 99)

5min
10min
15min
20min


In [13]:
# Calculating the correlation among different options of predictability measure and the set of prediction errors

correlation_dict = dict()

for entropy_type in entropy_df.columns:
    series_1 = entropy_df.loc[:,entropy_type].astype('float64')
    correlation_dict[entropy_type] = list()
    for forecast_error_type in forecast_error_df.columns:
        series_2 = forecast_error_df.loc[:,forecast_error_type].astype('float64')
        correlation_dict[entropy_type].append(series_1.corr(series_2))

In [ ]:
# Creating the correlation list of each predictability candidate to display in Figure 2

WPE3list, WPE4list, WPE5list, WPE6list, PE3list, PE4list, PE5list, PE6list, SpElist = [[] for _ in range(9)]

for key in correlation_dict.keys():
    if 'WPE3' in key:
        WPE3list.append(sum(correlation_dict[key])/len(correlation_dict[key]))
    elif 'WPE4' in key:
        WPE4list.append(sum(correlation_dict[key])/len(correlation_dict[key]))
    elif 'WPE5' in key:
        WPE5list.append(sum(correlation_dict[key])/len(correlation_dict[key]))
    elif 'WPE6' in key:
        WPE6list.append(sum(correlation_dict[key])/len(correlation_dict[key]))
    elif 'PE3' in key:
        PE3list.append(sum(correlation_dict[key])/len(correlation_dict[key]))
    elif 'PE4' in key:
        PE4list.append(sum(correlation_dict[key])/len(correlation_dict[key]))
    elif 'PE5' in key:
        PE5list.append(sum(correlation_dict[key])/len(correlation_dict[key]))
    elif 'PE6' in key:
        PE6list.append(sum(correlation_dict[key])/len(correlation_dict[key]))
    elif 'SpE' in key:
        SpElist.append(sum(correlation_dict[key])/len(correlation_dict[key]))

In [ ]:
# Plotting Figure 2

import matplotlib.pyplot as plt

plt.style.use('default')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']
plt.rcParams['font.size'] = 7
plt.rcParams['figure.dpi'] = 400
plt.rcParams['savefig.dpi'] = 400

cycler = plt.cycler(linestyle=['solid','dashdot', 'dotted', 'dashed', (0, (3, 1, 1, 1)), 'solid','dashdot', 'dotted', 'dashed'],
                    color=['purple','deepskyblue', 'pink', 'brown', 'darkorange', 'blueviolet', 'blue', 'darkgreen', 'red'],
                    )

fig = plt.figure(figsize=(3.4, 2))
ax = fig.add_subplot(111)
ax.set_prop_cycle(cycler)

lw_all = 0.7
ms_all = 1.4

plt.plot(resampling_interval_list, WPE3list, label = 'WPE (d=3)',
         lw = lw_all, marker='H', markersize=ms_all)
plt.plot(resampling_interval_list, WPE4list, label = 'WPE (d=4)',
         lw = lw_all, marker='^', markersize=ms_all)
plt.plot(resampling_interval_list, WPE5list, label = 'WPE (d=5)',
         lw = lw_all, marker='s', markersize=ms_all)
plt.plot(resampling_interval_list, WPE6list, label = 'WPE (d=6)',
         lw = lw_all, marker='o', markersize=ms_all)
plt.plot(resampling_interval_list, SpElist, label = 'SpE',
         lw = lw_all, marker='D', markersize=ms_all)
plt.plot(resampling_interval_list, PE3list, label = 'PE (d=3)',
         lw = lw_all, marker='<', markersize=ms_all)
plt.plot(resampling_interval_list, PE4list, label = 'PE (d=4)',
         lw = lw_all, marker='>', markersize=ms_all)
plt.plot(resampling_interval_list, PE5list, label = 'PE (d=5)',
         lw = lw_all, marker='P', markersize=ms_all)
plt.plot(resampling_interval_list, PE6list, label = 'PE (d=6)',
         lw = lw_all, marker='x', markersize=ms_all)

plt.xlabel('Resampling interval')
plt.ylabel("Average correlation")

plt.grid(color = 'gainsboro', linestyle = '--', linewidth = 0.4, alpha = 0.5)

plt.legend(bbox_to_anchor=(0, 1.02, 1, 0.2), loc="lower left", mode="expand", borderaxespad=0, ncol=3)

plt.savefig('Figure2.pdf',dpi = 400, bbox_inches='tight')

In [ ]:
# Plotting Figure 3

from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
from matplotlib.lines import Line2D

plt.style.use('default')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']
plt.rcParams['font.size'] = 8
plt.rcParams['figure.dpi'] = 400
plt.rcParams['savefig.dpi'] = 400


plt.figure(figsize=(6, 5.5))
plots = []

predictability = entropy_df['WPE6_10min'].astype('float64')

for i in range(4):
    for j in range(4):
        ax = plt.subplot2grid((4,4), (i,j))
        error = forecast_error_df.iloc[:,j*4 + i].astype('float64')
        if (i == 0) or (i == 2):
            selectedcolor = 'blue'
        elif (i == 1) or (i == 3):
            selectedcolor = 'darkorange'
        ax.scatter(predictability, error, s = 1, alpha = 0.4, color = selectedcolor)

        X2 = sm.add_constant(predictability)
        est = sm.OLS(error, X2).fit()

        B = est.params[0]
        A = est.params[1]
        C = est.params[0] - 2*est.bse[0]
        D = est.params[0] + 2*est.bse[0]
        xx = np.linspace(0.66,0.88 ,100)
        Regress = A*xx+B
        plt.grid(color = 'gainsboro', axis = 'both', linestyle = '--', linewidth = 0.4, alpha = 0.5)
        ax.plot(xx, Regress, '-', color = 'red',  alpha = 0.6 )

        anchored_text = AnchoredText('$R^2$=%0.2f' % est.rsquared, loc=2, pad = 0.12, frameon = 0)
        ax.add_artist(anchored_text)

        if i == 0:
            plt.ylim([0.004, 0.063])
            plt.ylabel('NMAE')
            ax.set_yticks([0.02,0.04, 0.06])
            if j == 0:
                ax.set_title('5-min ahead')
            elif j == 1:
                ax.set_title('10-min ahead')
            elif j == 2:
                ax.set_title('15-min ahead')
            elif j == 3:
                ax.set_title('20-min ahead')
        elif i == 1:
            plt.ylim([0.004, 0.063])
            plt.ylabel('NMAE')
            ax.set_yticks([0.02,0.04, 0.06])
        elif i == 2:
            plt.ylim([0.03, 0.14])
            plt.ylabel('NRMSE')
            ax.set_yticks([0.04,0.08, 0.12])
        elif i == 3:
            plt.ylim([0.03, 0.14])
            plt.ylabel('NRMSE')
            ax.set_yticks([0.04,0.08, 0.12])
        plt.xlabel('WPE')
        plt.xlim([0.62, 0.91])

        if i != 3:
            plt.xlabel('')
            ax.axes.xaxis.set_ticklabels([])
        if j != 0:
            plt.ylabel('')
            ax.axes.yaxis.set_ticklabels([])
            plt.tick_params(axis='y', which='both', left=False)
        plt.tick_params(axis='y', which='both', labelbottom=False)

plt.subplots_adjust(wspace=0, hspace=0)

legend_elements = [Line2D([0], [0], marker="o", markersize=2, color='blue', linewidth=0, markeredgewidth=0.4,
                          alpha=0.9, markerfacecolor='blue', label='ARIMA'),
                   Line2D([0], [0], marker="o", markersize=2, color='darkorange', linewidth=0, markeredgewidth=0.4,
                          alpha=0.9, markerfacecolor='darkorange', label='naïve'),
                   Line2D([0], [0], color='red',  alpha = 0.6, label='Regressed line')]

plt.legend(handles=legend_elements, ncol = 3, loc =2, bbox_to_anchor=(-2.2, 4.5))

plt.savefig('Figure3.pdf',dpi = 400, bbox_inches='tight')

In [4]:
# Calculates the median/mean of day-ahead prediction errors of PV generation data for day ahead prediction

train_days = 9
test_days = 1

NRMSE_RF_df = pd.read_pickle('Processed_data/RF_' + str(train_days) + 'train_' + str(test_days) + 'test_NRMSE.pkl')
NMAE_RF_df = pd.read_pickle('Processed_data/RF_' + str(train_days) + 'train_' + str(test_days) + 'test_NMAE.pkl')
NRMSE_naive_df = pd.read_pickle('Processed_data/naive_' + str(train_days) + 'train_' + str(test_days) + 'test_NRMSE.pkl')
NMAE_naive_df = pd.read_pickle('Processed_data/naive_' + str(train_days) + 'train_' + str(test_days) + 'test_NMAE.pkl')

day_ahead_forecast_error_df = pd.DataFrame()
day_ahead_forecast_error_df.index = all_sites

for site_id in all_sites:
    day_ahead_forecast_error_df.loc[site_id,'RF_NMAE'] = NMAE_RF_df[site_id].median()
    day_ahead_forecast_error_df.loc[site_id,'naive_NMAE'] = NMAE_naive_df[site_id].median()
    day_ahead_forecast_error_df.loc[site_id,'RF_NRMSE'] = NRMSE_RF_df[site_id].median()
    day_ahead_forecast_error_df.loc[site_id,'naive_NRMSE'] = NRMSE_naive_df[site_id].median()

In [ ]:
# Plotting Figure 4

from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
from matplotlib.lines import Line2D

plt.style.use('default')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']
plt.rcParams['font.size'] = 8
plt.rcParams['figure.dpi'] = 400
plt.rcParams['savefig.dpi'] = 400

plt.figure(figsize=(3.35, 3.35))
plots = []
predictability = entropy_df['WPE6_10min'].astype('float64')

for i in range(2):
    for j in range(2):
        ax = plt.subplot2grid((2,2), (i,j))
        error = day_ahead_forecast_error_df.iloc[:,j + i*2].astype('float64')
        if (j == 0):
            selectedcolor = 'darkviolet'
        elif (j == 1):
            selectedcolor = 'darkorange'
        ax.scatter(predictability, error, s = 1, alpha = 0.4, color = selectedcolor)

        X2 = sm.add_constant(predictability)
        est = sm.OLS(error, X2).fit()

        B = est.params[0]
        A = est.params[1]
        C = est.params[0] - 2*est.bse[0]
        D = est.params[0] + 2*est.bse[0]
        xx = np.linspace(0.66,0.88 ,100)
        Regress = A*xx+B
        plt.grid(color = 'gainsboro', axis = 'both', linestyle = '--', linewidth = 0.5, alpha = 0.5)
        ax.plot(xx, Regress, '-', color = 'olive',  alpha = 0.6 )

        anchored_text = AnchoredText('$R^2$=%0.2f' % est.rsquared, loc=2, pad = 0.12, frameon = 0)
        ax.add_artist(anchored_text)

        if i == 0:
            plt.ylim([0.027, 0.12])
            plt.ylabel('median NMAE')
            ax.set_yticks([0.04,0.07, 0.1])
        elif i == 1:
            plt.ylim([0.07, 0.22])
            plt.ylabel('median NRMSE')

        plt.xlabel('WPE')
        plt.xlim([0.62, 0.91])

        if i != 1:
            plt.xlabel('')
            ax.axes.xaxis.set_ticklabels([])
        if j != 0:
            plt.ylabel('')
            ax.axes.yaxis.set_ticklabels([])
            plt.tick_params(axis='y', which='both', left=False)
        plt.tick_params(axis='y', which='both', labelbottom=False)

plt.subplots_adjust(wspace=0, hspace=0)

legend_elements = [Line2D([0], [0], marker="o", markersize=2, color='darkviolet', linewidth = 0, markeredgewidth = 0.4,
                          alpha = 0.9, markerfacecolor='darkviolet', label='Random Forest'),
                   Line2D([0], [0], marker="o", markersize=2, color='darkorange', linewidth = 0, markeredgewidth = 0.4,
                          alpha = 0.9, markerfacecolor='darkorange', label='Seasonal naïve'),
                   Line2D([0], [0], color='olive', alpha = 0.6, label='Regressed line')]

plt.legend(handles=legend_elements, ncol = 2, loc =2, bbox_to_anchor=(-1, 2.35))

plt.savefig('Figure4.pdf',dpi = 400, bbox_inches='tight')